## 6 Short Write Up
* The following code tries to firstly get unigram and bigram counts of the sentence. Then calculates the partial derivative and finite difference based on the loglikelihood discussed in the class.
* The h value used is 0.000001
* The code is almost self explanatory and well organized.
* Since there is no mention of explaining the code or submission of the code, there is no detailed explanation of the code being given.
* As required two csv files are submitted in the requested format.

In [1]:
def get_lines_from_file(filename):
    f=open(filename,encoding="utf-8")
    lines=[x.strip() for x in f]
    return lines

In [2]:
lines = get_lines_from_file("../en_ewt-ud-train.chars.txt")

In [3]:
observed_count={}
for i in lines[0]:
    if not observed_count.keys():
        observed_count[i]=1
    elif i not in observed_count.keys():
        observed_count[i]=1
    else:
        observed_count[i]+=1

In [4]:
#observed_count['B']=1

observed_count['U']=0
observed_count_dummy=observed_count.copy()

observed_count['E']=1
print(len(observed_count))

39


In [5]:
bigram_count={}

observed_count_dummy['B']=1
for i in observed_count_dummy.keys():
    for j in observed_count.keys():
        bigram_count[(i,j)]=0
prev='B'
lines[0]=lines[0]+"E"
for i in lines[0]:
    if not bigram_count.keys():
        bigram_count[(prev,i)]=1
    elif (prev,i) not in bigram_count.keys():
        bigram_count[(prev,i)]=1
    else:
        bigram_count[(prev,i)]+=1
    prev=i

In [6]:
import numpy as np
observed_feature_count=observed_count.copy()
expected_feature_count=observed_count.copy()
for key,value in expected_feature_count.items():
    expected_feature_count[key]=(1/len(observed_count.keys()))*sum(observed_count.values())

In [7]:
partial_derivative={}
for key,value in observed_count.items():
    partial_derivative[key]=observed_count[key]-expected_feature_count[key]

In [8]:
import numpy as np
finite_difference={}
for superkey in observed_count.keys():
    sum1=0
    sum2=0
    denom1=len(observed_count.keys())-1+np.exp(0.000001)
    denom2=len(observed_count.keys())-1+np.exp(-0.000001)
    for key in observed_count.keys():
        if key!=superkey:
            sum1+=np.log(1/denom1)*observed_count[key]
            sum2+=np.log(1/denom2)*observed_count[key]
        else:
            sum1+=np.log(np.exp(0.000001)/denom1)*observed_count[key]
            sum2+=np.log(np.exp(-0.000001)/denom2)*observed_count[key]
    finite_difference[superkey]=(sum1-sum2)/0.000002
#finite_difference

In [9]:
expected_feature_count_2=observed_count.copy()
norm=0
for key in expected_feature_count_2.keys():
    norm+=np.exp(np.tanh((ord(key)-74)/128))
for key,value in expected_feature_count_2.items():
    prob=np.exp(np.tanh((ord(key)-74)/128))/norm
    expected_feature_count_2[key]=prob*sum(observed_count.values())

In [10]:
partial_derivative_2={}
for key,value in observed_count.items():
    partial_derivative_2[key]=observed_count[key]-expected_feature_count_2[key]

In [11]:
import numpy as np
finite_difference_2={}
norm=0
for key in observed_count.keys():
        norm+=np.exp(np.tanh((ord(key)-74)/128))
        
for superkey in observed_count.keys():
    sum1=0
    sum2=0
    
    denom1=norm-np.exp(np.tanh((ord(superkey)-74)/128))
    denom1+=np.exp(np.tanh((ord(superkey)-74)/128)+0.000001)
    
    denom2=norm-np.exp(np.tanh((ord(superkey)-74)/128))
    denom2+=np.exp(np.tanh((ord(superkey)-74)/128)-0.000001)
    
    for key in observed_count.keys():
        if key!=superkey:
            temp=np.exp(np.tanh((ord(key)-74)/128))
            prob=temp/denom1
            sum1+=np.log(prob)*observed_count[key]
            prob=temp/denom2
            sum2+=np.log(prob)*observed_count[key]
        else:
            temp=np.exp(np.tanh((ord(key)-74)/128)+0.000001)
            prob=temp/denom1
            sum1+=np.log(prob)*observed_count[key]
            temp=np.exp(np.tanh((ord(key)-74)/128)-0.000001)
            prob=temp/denom2
            sum2+=np.log(prob)*observed_count[key]
    
    finite_difference_2[superkey]=(sum1-sum2)/0.000002
#finite_difference_2

In [12]:
import pandas as pd
features={}
for key in observed_count.keys():
    features[key]=key
features[" "]="SPACE"
df = pd.DataFrame([features, observed_count]).T
df.columns=["features", "observed_count"]

In [13]:
#New code
df["features"]=features.values()
df["observed_count"]=observed_count.values()

In [14]:
df["expected_feature_count_theta_0"]=expected_feature_count.values()
df["partial_derivative_theta_0"]=partial_derivative.values()
df["finite_difference_theta_0"]=finite_difference.values()
df["expected_feature_count_theta_tanh"]=expected_feature_count_2.values()
df["partial_derivative_theta_tanh"]=partial_derivative_2.values()
df["finite_difference_theta_tanh"]=finite_difference_2.values()

In [15]:
df.to_csv("unigram_maxent_results.csv")

In [16]:
for key in observed_count_dummy.keys():
    observed_count_dummy[key]=0
    
for key,value in bigram_count.items():
    i,j=key
    observed_count_dummy[i]+=value

In [17]:
expected_bigram_counts=bigram_count.copy()
expected_unigram_count=expected_feature_count.copy()
for key,value in bigram_count.items():
    i,j=key
    expected_bigram_counts[key]=(1/39)*observed_count_dummy[i]

In [18]:
partial_derivative_bigrams={}
for key,value in expected_bigram_counts.items():
    partial_derivative_bigrams[key]=bigram_count[key]-expected_bigram_counts[key]

In [19]:
import numpy as np
finite_difference_bigram={}
for superkey in bigram_count.keys():
    I,J=superkey
    sum1=0
    sum2=0
    denom1=len(observed_count_dummy.keys())-1+np.exp(0.00001)
    denom2=len(observed_count_dummy.keys())-1+np.exp(-0.00001)
    for key in bigram_count.keys():
        i,j=key
        if i!=I:
            continue
        if key!=superkey:
            sum1+=np.log(1/denom1)*bigram_count[key]
            sum2+=np.log(1/denom2)*bigram_count[key]
        else:
            sum1+=np.log(np.exp(0.00001)/denom1)*bigram_count[key]
            sum2+=np.log(np.exp(-0.00001)/denom2)*bigram_count[key]
    finite_difference_bigram[superkey]=(sum1-sum2)/0.00002
#finite_difference

In [20]:
expected_bigram_counts_2=bigram_count.copy()
expected_unigram_count_2=expected_feature_count_2.copy()

for key,value in bigram_count.items():
    i,j=key
    norm=0
    for sing_key in observed_count_dummy.keys():
        norm+=np.exp(np.tanh((ord(sing_key)-ord(i))/128))
    prob=np.exp(np.tanh((ord(j)-ord(i))/128))/norm
    expected_bigram_counts_2[key]=prob*observed_count_dummy[i]

In [21]:
partial_derivative_bigrams_2={}
for key,value in expected_bigram_counts.items():
    partial_derivative_bigrams_2[key]=bigram_count[key]-expected_bigram_counts_2[key]

In [22]:
import numpy as np
finite_difference_bigram_2={}
norm=0
        
for superkey in bigram_count.keys():
    I,J=superkey
    sum1=0
    sum2=0
    norm=0
    for sing_key in observed_count_dummy.keys():
        norm+=np.exp(np.tanh((ord(sing_key)-ord(I))/128))
        
    denom1=norm-np.exp(np.tanh((ord(J)-ord(I))/128))
    denom1+=np.exp(np.tanh((ord(J)-ord(I))/128)+0.00001)
    
    denom2=norm-np.exp(np.tanh((ord(J)-ord(I))/128))
    denom2+=np.exp(np.tanh((ord(J)-ord(I))/128)-0.00001)
    
    for key in bigram_count.keys():
        i,j=key
        if i!=I:
            continue
        if key!=superkey:
            temp=np.exp(np.tanh((ord(j)-ord(I))/128))
            prob=temp/denom1
            sum1+=np.log(prob)*bigram_count[key]
            prob=temp/denom2
            sum2+=np.log(prob)*bigram_count[key]
        else:
            temp=np.exp(np.tanh((ord(j)-ord(I))/128)+0.00001)
            prob=temp/denom1
            sum1+=np.log(prob)*bigram_count[key]
            temp=np.exp(np.tanh((ord(j)-ord(I))/128)-0.00001)
            prob=temp/denom2
            sum2+=np.log(prob)*bigram_count[key]
    
    finite_difference_bigram_2[superkey]=(sum1-sum2)/0.00002
#finite_difference_bigram_2

In [23]:
import pandas as pd
features={}
for key in bigram_count.keys():
    i=key[0]
    j=key[1]
    if key[0]==" ":
        i="SPACE"
    if key[1]==" ":
        j="SPACE"
    features[key]=(i,j)
#for key in observed_count_dummy.keys():
#    features[key]=key
df_2 = pd.DataFrame([features, bigram_count]).T
df_2.columns=["features", "observed_count"]

In [24]:
#New code
df_2["features"]=features.values()
df_2["observed_count"]=bigram_count.values()

In [25]:
df_2["expected_feature_count_theta_0"]=expected_bigram_counts.values()
df_2["partial_derivative_theta_0"]=partial_derivative_bigrams.values()
df_2["finite_difference_theta_0"]=finite_difference_bigram.values()
df_2["expected_feature_count_theta_tanh"]=expected_bigram_counts_2.values()
df_2["partial_derivative_theta_tanh"]=partial_derivative_bigrams_2.values()
df_2["finite_difference_theta_tanh"]=finite_difference_bigram_2.values()

In [26]:
df_2=df_2.append(df)
#df_2

In [27]:
df_2.to_csv("bigram_maxent_results.csv")